## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [1]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [2]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'

# Range to read (change this for each project)
start_row = 8
end_row = 65
usecols = 'A:AF'

print("Variables defined correctly!")

Variables defined correctly!


## Extract Data from "Informe de métricas históricas" CSV file
Extract data from the CSV file and convert it directly to a DataFrame.

In [3]:
import pandas as pd

# Asumiendo que ya has cargado y pivotado el DataFrame
df = pd.read_csv(csv_file_path)

pivot_df = df.pivot_table(index=['Agent', 'StartInterval', 'EndInterval'], 
                          values=['Contacts handled incoming', 'Contacts transferred out internal'], 
                          aggfunc='first')

# Verificar los niveles del índice
print("Niveles del índice:", pivot_df.index.names)

# Verificar las columnas del DataFrame
print("Columnas del DataFrame:", pivot_df.columns)

# Mostrar las primeras filas del DataFrame para verificar el contenido
print("Contenido del DataFrame:")
print(pivot_df.head())

# Verificar el tipo de datos del DataFrame
print("Tipo de datos:", type(pivot_df))

# Verificar los valores únicos en cada nivel del índice
print("Agentes únicos:", pivot_df.index.get_level_values('Agent').unique())
print("StartInterval únicos:", pivot_df.index.get_level_values('StartInterval').unique())
print("EndInterval únicos:", pivot_df.index.get_level_values('EndInterval').unique())



Niveles del índice: ['Agent', 'StartInterval', 'EndInterval']
Columnas del DataFrame: Index(['Contacts handled incoming', 'Contacts transferred out internal'], dtype='object')
Contenido del DataFrame:
                                                                                                Contacts handled incoming  \
Agent                              StartInterval                 EndInterval                                                
Irati.izaguirre@originaltelecom.es 2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       19.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       23.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       38.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       43.0   
                                   2024-08-07T00:

## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [4]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Pivot Table Data

Verify that all agents are present in the Pivot Table. If an agent is missing, add a row with zeros for that agent.

In [5]:
def remove_extra_agents_from_pivot(pivot_df, agents_list):
    """
    Remove agents from the pivot DataFrame that are not in the agents_list.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to keep.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with only the agents in agents_list.
    """
    # Verificar si 'Agent' es un nivel de índice
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']
    
    # Filtrar el índice para mantener solo los agentes que están en agents_list
    mask = pivot_df.index.get_level_values('Agent').isin(agents_list)
    pivot_df = pivot_df[mask]
    
    # Verificar si se han eliminado los agentes no deseados
    remaining_unwanted_agents = [agent for agent in pivot_df.index.get_level_values('Agent').unique() if "formacion" in agent]
    if remaining_unwanted_agents:
        print(f"Los siguientes agentes no deseados siguen presentes: {remaining_unwanted_agents}")
    else:
        print("Todos los agentes no deseados han sido eliminados correctamente.")
    
    return pivot_df

# Asegúrate de que 'agents_list' no contiene agentes de "formación"
agents_list = [agent for agent in agents_list if "formacion" not in agent]

# Aplicar la función a la tabla pivoteada
pivot_df = remove_extra_agents_from_pivot(pivot_df, agents_list)

# Mostrar los agentes únicos después de filtrar
unique_agents = pivot_df.index.get_level_values('Agent').unique()
print("Agentes únicos después de filtrar:", unique_agents.tolist())


def add_missing_agents(pivot_df, agents_list):
    """
    Add missing agents to the pivot DataFrame with zero values for relevant columns.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to ensure are present.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with all agents.
    """
    # Verificar si 'Agent' es un nivel de índice, si no, restaurarlo
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']

    # Obtener la lista de agentes presentes en el DataFrame pivoteado
    existing_agents = pivot_df.index.get_level_values('Agent').unique()

    # Identificar agentes faltantes que están en la lista pero no en el pivot_df
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]

    # Crear combinaciones de 'StartInterval' y 'EndInterval'
    date_combinations = list(itertools.product(
        pivot_df.index.get_level_values('StartInterval').unique(),
        pivot_df.index.get_level_values('EndInterval').unique()
    ))

    # Añadir filas para los agentes faltantes con valores cero
    for agent in missing_agents:
        for start_interval, end_interval in date_combinations:
            zero_row = pd.Series(
                [0, 0],  # Valores para 'Contacts handled incoming' y 'Contacts transferred out internal'
                index=pivot_df.columns,
                name=(agent, start_interval, end_interval)
            )
            pivot_df = pd.concat([pivot_df, zero_row.to_frame().T])

    print(f"Added {len(missing_agents)} missing agents.")
    return pivot_df

# Aplicar las funciones a la tabla pivoteada
informe_df = remove_extra_agents_from_pivot(pivot_df, agents_list)
informe_df = add_missing_agents(informe_df, agents_list)

# Mostrar el DataFrame actualizado
print("Informe DataFrame after ensuring all agents:")
display(informe_df.head(55))

# Verificar los niveles del índice
print("Niveles del índice:", pivot_df.index.names)

# Verificar las columnas del DataFrame
print("Columnas del DataFrame:", pivot_df.columns)

# Mostrar las primeras filas del DataFrame para verificar el contenido
print("Contenido del DataFrame:")
print(pivot_df.head())

# Verificar el tipo de datos del DataFrame
print("Tipo de datos:", type(pivot_df))

# Verificar los valores únicos en cada nivel del índice
# Mostrar todos los agentes únicos en el DataFrame
unique_agents = pivot_df.index.get_level_values('Agent').unique()

# Mostrar todos los agentes únicos
print("Agentes únicos:", unique_agents.tolist())
print("StartInterval únicos:", pivot_df.index.get_level_values('StartInterval').unique())
print("EndInterval únicos:", pivot_df.index.get_level_values('EndInterval').unique())



Todos los agentes no deseados han sido eliminados correctamente.
Agentes únicos después de filtrar: ['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'mar.marchena@originaltelecom.es', 'maria.torres@originaltelecom.es', 'mariaarroyo@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'marta.dorado@originaltelecom

Contacts handled incoming  \
albaaraujo@originaltelecom.es      2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       58.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       59.0   
albertocanto@originaltelecom.es    2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       36.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       16.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       64.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       42.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       35.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       41.0   
albertosanchez@originaltelecom.es  2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       41.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       27.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       45.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       44.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       39.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       53.0   
anasanchez@originaltelecom.es      2024-08-03T00:00:00.000+02:00 2024-08-04T00:00:00.000+02:00                       55.0   
                                   2024-08-04T00:00:00.000+02:00 2024-08-05T00:00:00.000+02:00                       24.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       50.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       33.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       45.0   
antonio.reina@originaltelecom.es   2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       30.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       23.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       58.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       53.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       33.0   
                                   2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02:00                       31.0   
beatriz.gomez@originaltelecom.es   2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       26.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       28.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00  

Niveles del índice: ['Agent', 'StartInterval', 'EndInterval']
Columnas del DataFrame: Index(['Contacts handled incoming', 'Contacts transferred out internal'], dtype='object')
Contenido del DataFrame:
                                                                                           Contacts handled incoming  \
Agent                         StartInterval                 EndInterval                                                
albaaraujo@originaltelecom.es 2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                              2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                              2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                              2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                              2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00

## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [7]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Ordenar la tabla pivoteada por valores numéricos asociados a los correos electrónicos
def sort_pivot_table_by_email_values(pivot_df, email_values):
    # Crear una serie temporal con los valores de email
    temp_index = pivot_df.index.get_level_values('Agent').map(email_values)
    
    # Crear un nuevo DataFrame temporal con el índice original y los valores numéricos
    temp_df = pd.DataFrame({'temp_index': temp_index, 'original_index': pivot_df.index})
    
    # Ordenar el DataFrame temporal por los valores numéricos
    temp_df = temp_df.sort_values(by='temp_index')
    
    # Reordenar la tabla pivoteada usando el índice original ordenado
    pivot_df = pivot_df.loc[temp_df['original_index']]
    
    return pivot_df

# Aplicar la función para ordenar la tabla pivoteada
sorted_pivot_df = sort_pivot_table_by_email_values(pivot_df, email_values)

# Mostrar la tabla pivoteada ordenada
print("Informe DataFrame sorted by email values:")
print(sorted_pivot_df.head(15))
print("Niveles del índice:", pivot_df.index.names)

Informe DataFrame sorted by email values:
                                                                                               Contacts handled incoming  \
Agent                             StartInterval                 EndInterval                                                
albaaraujo@originaltelecom.es     2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       33.0   
                                  2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       29.0   
                                  2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       63.0   
                                  2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       46.0   
                                  2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       58.0   
                                  2024-08-08T00:00:00.000+02:00 2024-08-09T00:00:00.000+02

## Merge StartInterval and EndInterval

In [8]:

# Supongamos que pivot_df es tu DataFrame pivoteado con tres niveles de índice

# Combinar 'StartInterval' y 'EndInterval' en un solo índice de cadena
# Aquí se usa ' - ' para separar las fechas, pero puedes elegir otro separador
new_index = pivot_df.index.map(lambda x: f"{x[1]} - {x[2]}")

# Asignar el nuevo índice al DataFrame, manteniendo 'Agent' como el primer nivel de índice
pivot_df.set_index(new_index, append=True, inplace=True)

# Eliminar los índices 'StartInterval' y 'EndInterval' originales
pivot_df.reset_index(level=['StartInterval', 'EndInterval'], drop=True, inplace=True)

# Renombrar el nuevo nivel de índice para mayor claridad
pivot_df.index.names = ['Agent', 'DateInterval']

# Mostrar el DataFrame con el nuevo índice
print(pivot_df)

print("Niveles del índice:", pivot_df.index.names)


                                                                                      Contacts handled incoming  \
Agent                             DateInterval                                                                    
albaaraujo@originaltelecom.es     2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       33.0   
                                  2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       29.0   
                                  2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       63.0   
                                  2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       46.0   
                                  2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       58.0   
...                                                                                                         ...   
yicel.patricia@originaltelecom.es 2024-08-04T00:00:00.000+02:00 - 2024-08-05T00:

## Checking

In [ ]:
print(pivot_df.index.names)
if 'Agent' not in pivot_df.index.names or 'DateInterval' not in pivot_df.index.names:
    pivot_df.set_index(['Agent', 'DateInterval'], inplace=True)



## Add missing Intervals

In [10]:
import itertools

def add_missing_date_intervals(pivot_df, agents_list, complete_date_intervals):
    """
    Ensure that all agents have all date intervals present in the pivot DataFrame.
    If any date interval is missing for an agent, add it with zero values.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to ensure are present.
    complete_date_intervals (list): List of all date intervals that should be present.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with all agents and date intervals.
    """
    # Verificar si 'Agent' y 'DateInterval' son niveles de índice
    if 'Agent' not in pivot_df.index.names or 'DateInterval' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'DateInterval']
    
    # Añadir filas para los intervalos de fecha faltantes con valores cero
    for agent in agents_list:
        for date_interval in complete_date_intervals:
            if (agent, date_interval) not in pivot_df.index:
                # Agregar fila con valores cero
                zero_row = pd.Series(
                    [0, 0],  # Valores para 'Contacts handled incoming' y 'Contacts transferred out internal'
                    index=pivot_df.columns,
                    name=(agent, date_interval)
                )
                pivot_df = pd.concat([pivot_df, zero_row.to_frame().T])

    print("Added missing date intervals for all agents.")
    return pivot_df

# Lista completa de intervalos de fecha requeridos
complete_date_intervals = [
    '2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:00:00.000+02:00',
    '2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:00:00.000+02:00',
    '2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:00:00.000+02:00',
    '2024-08-04T00:00:00.000+02:00 - 2024-08-05T00:00:00.000+02:00',
    '2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:00:00.000+02:00',
    '2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:00:00.000+02:00',
    '2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:00:00.000+02:00',
    '2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:00:00.000+02:00',
    '2024-08-09T00:00:00.000+02:00 - 2024-08-10T00:00:00.000+02:00',
    '2024-08-10T00:00:00.000+02:00 - 2024-08-11T00:00:00.000+02:00',
    '2024-08-11T00:00:00.000+02:00 - 2024-08-12T00:00:00.000+02:00',
    '2024-08-12T00:00:00.000+02:00 - 2024-08-13T00:00:00.000+02:00',
    '2024-08-13T00:00:00.000+02:00 - 2024-08-14T00:00:00.000+02:00',
    '2024-08-14T00:00:00.000+02:00 - 2024-08-15T00:00:00.000+02:00',
    '2024-08-15T00:00:00.000+02:00 - 2024-08-16T00:00:00.000+02:00',
    '2024-08-16T00:00:00.000+02:00 - 2024-08-17T00:00:00.000+02:00',
    '2024-08-17T00:00:00.000+02:00 - 2024-08-18T00:00:00.000+02:00',
    '2024-08-18T00:00:00.000+02:00 - 2024-08-19T00:00:00.000+02:00',
    '2024-08-19T00:00:00.000+02:00 - 2024-08-20T00:00:00.000+02:00',
    '2024-08-20T00:00:00.000+02:00 - 2024-08-21T00:00:00.000+02:00',
    '2024-08-21T00:00:00.000+02:00 - 2024-08-22T00:00:00.000+02:00',
    '2024-08-22T00:00:00.000+02:00 - 2024-08-23T00:00:00.000+02:00',
    '2024-08-23T00:00:00.000+02:00 - 2024-08-24T00:00:00.000+02:00',
    '2024-08-24T00:00:00.000+02:00 - 2024-08-25T00:00:00.000+02:00',
    '2024-08-25T00:00:00.000+02:00 - 2024-08-26T00:00:00.000+02:00',
    '2024-08-26T00:00:00.000+02:00 - 2024-08-27T00:00:00.000+02:00',
    '2024-08-27T00:00:00.000+02:00 - 2024-08-28T00:00:00.000+02:00',
    '2024-08-28T00:00:00.000+02:00 - 2024-08-29T00:00:00.000+02:00',
    '2024-08-29T00:00:00.000+02:00 - 2024-08-30T00:00:00.000+02:00',
    '2024-08-30T00:00:00.000+02:00 - 2024-08-31T00:00:00.000+02:00',
    '2024-08-31T00:00:00.000+02:00 - 2024-09-01T00:00:00.000+02:00'
]

# Aplicar la función para añadir los intervalos de fechas que faltan
pivot_df = add_missing_date_intervals(pivot_df, agents_list, complete_date_intervals)

# Mostrar el DataFrame actualizado
print("Informe DataFrame after adding missing date intervals:")
display(pivot_df.head(55))
print(pivot_df.loc[('albaaraujo@originaltelecom.es', '2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:00:00.000+02:00')])
print(f"Total de filas en la pivot table: {pivot_df.shape[0]}")
agent_specific_df = pivot_df.loc['albaaraujo@originaltelecom.es']
print(agent_specific_df)
print("Comprobando si existen filas ocultas debido a filtros:")
print(pivot_df[pivot_df.isna().any(axis=1)])
print("Niveles del índice:", pivot_df.index.names)


Added missing date intervals for all agents.
Informe DataFrame after adding missing date intervals:


Contacts handled incoming  \
Agent                              DateInterval                                                                    
albaaraujo@originaltelecom.es      2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       33.0   
                                   2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       29.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       63.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       46.0   
                                   2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       58.0   
                                   2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:0...                       59.0   
albertocanto@originaltelecom.es    2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       36.0   
                                   2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       16.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       64.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       42.0   
                                   2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       35.0   
                                   2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:0...                       41.0   
albertosanchez@originaltelecom.es  2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       41.0   
                                   2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       27.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       45.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       44.0   
                                   2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       39.0   
                                   2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:0...                       53.0   
anasanchez@originaltelecom.es      2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:0...                       55.0   
                                   2024-08-04T00:00:00.000+02:00 - 2024-08-05T00:0...                       24.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       50.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       33.0   
                                   2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       45.0   
antonio.reina@originaltelecom.es   2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       30.0   
                                   2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       23.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       58.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       53.0   
                                   2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       33.0   
                                   2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:0...                       31.0   
beatriz.gomez@originaltelecom.es   2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       26.0   
                                   2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       28.0   
                                   2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       40.0   
                                   2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       31.0   
                           

Contacts handled incoming            0.0
Contacts transferred out internal    0.0
Name: (albaaraujo@originaltelecom.es, 2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:00:00.000+02:00), dtype: float64
Total de filas en la pivot table: 1333
                                                    Contacts handled incoming  \
DateInterval                                                                    
2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:0...                       33.0   
2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:0...                       29.0   
2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:0...                       63.0   
2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:0...                       46.0   
2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:0...                       58.0   
2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:0...                       59.0   
2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:0...                        0.0   
2024-08-04T00:00:00.000+02:00 - 2

## Update Final Sheet

In [12]:


final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'DIARIO AGOSTO'

def update_excel_from_pivot(pivot_df, file_path):
    """
    Update an Excel sheet with values from a pivot table based on agent and date intervals.

    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook.active  # Adjust if you need a specific sheet

    # Define the column mappings for DateIntervals
    date_interval_columns = [
        ('F', 'G'), ('K', 'L'), ('P', 'Q'), ('U', 'V'), ('Z', 'AA'),
        ('AE', 'AF'), ('AJ', 'AK'), ('AO', 'AP'), ('AT', 'AU'), ('AY', 'AZ'),
        ('BD', 'BE'), ('BI', 'BJ'), ('BN', 'BO'), ('BS', 'BT'), ('BX', 'BY'),
        ('CC', 'CD'), ('CH', 'CI'), ('CM', 'CN'), ('CR', 'CS'), ('CW', 'CX'),
        ('DB', 'DC'), ('DG', 'DH'), ('DL', 'DM'), ('DQ', 'DR'), ('DV', 'DW'),
        ('EA', 'EB'), ('EF', 'EG'), ('EK', 'EL'), ('EP', 'EQ'), ('EU', 'EV'),
        ('EZ', 'FA')
    ]

    # Recorrer cada agente y actualizar el Excel
    for agent_index, (agent, agent_data) in enumerate(pivot_df.groupby(level='Agent')):
        row = agent_index + 4  # Fila basada en el número de agente + 3

        for interval_index, (date_interval, data) in enumerate(agent_data.groupby(level='DateInterval')):
            # Columnas correspondientes al intervalo de fecha actual
            col_incoming, col_transferred = date_interval_columns[interval_index]

            # Obtener los valores a insertar
            incoming_value = data['Contacts handled incoming'].values[0]
            transferred_value = data['Contacts transferred out internal'].values[0]

            # Actualizar las celdas en el Excel
            sheet[f'{col_incoming}{row}'] = incoming_value
            sheet[f'{col_transferred}{row}'] = transferred_value

    workbook.save(file_path)
    print(f"Excel updated successfully with data from pivot table.")

# Uso del código
update_excel_from_pivot(pivot_df, final_file_path)



Excel updated successfully with data from pivot table.


## Update test

In [13]:
from openpyxl import load_workbook

def update_excel_from_pivot(pivot_df, file_path):
    """
    Update an Excel sheet with values from a pivot table based on agent and date intervals.

    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']  # Específico para la hoja 'DIARIO AGOSTO'

    # Define the column mappings for DateIntervals
    date_interval_columns = [
        ('F', 'G'), ('K', 'L'), ('P', 'Q'), ('U', 'V'), ('Z', 'AA'),
        ('AE', 'AF'), ('AJ', 'AK'), ('AO', 'AP'), ('AT', 'AU'), ('AY', 'AZ'),
        ('BD', 'BE'), ('BI', 'BJ'), ('BN', 'BO'), ('BS', 'BT'), ('BX', 'BY'),
        ('CC', 'CD'), ('CH', 'CI'), ('CM', 'CN'), ('CR', 'CS'), ('CW', 'CX'),
        ('DB', 'DC'), ('DG', 'DH'), ('DL', 'DM'), ('DQ', 'DR'), ('DV', 'DW'),
        ('EA', 'EB'), ('EF', 'EG'), ('EK', 'EL'), ('EP', 'EQ'), ('EU', 'EV'),
        ('EZ', 'FA')
    ]

    # Recorrer cada agente y actualizar el Excel
    for agent_index, (agent, agent_data) in enumerate(pivot_df.groupby(level='Agent')):
        row = agent_index + 4  # Fila basada en el número de agente + 3

        # Comprobamos que estamos trabajando con el agente correcto
        print(f"Actualizando datos para el agente: {agent} en la fila {row}")

        for interval_index, (date_interval, data) in enumerate(agent_data.groupby(level='DateInterval')):
            # Columnas correspondientes al intervalo de fecha actual
            col_incoming, col_transferred = date_interval_columns[interval_index]

            # Obtener los valores a insertar
            incoming_value = data['Contacts handled incoming'].values[0]
            transferred_value = data['Contacts transferred out internal'].values[0]

            # Comprobación de valores antes de actualizar
            print(f"Fecha: {date_interval} -> Insertando en columnas {col_incoming} y {col_transferred}, valores: {incoming_value}, {transferred_value}")

            # Actualizar las celdas en el Excel
            sheet[f'{col_incoming}{row}'] = incoming_value
            sheet[f'{col_transferred}{row}'] = transferred_value

    workbook.save(file_path)
    print(f"Excel updated successfully with data from pivot table.")

# Uso del código
update_excel_from_pivot(pivot_df, final_file_path)

Actualizando datos para el agente: albaaraujo@originaltelecom.es en la fila 4
Fecha: 2024-08-01T00:00:00.000+02:00 - 2024-08-02T00:00:00.000+02:00 -> Insertando en columnas F y G, valores: 33.0, 18.0
Fecha: 2024-08-02T00:00:00.000+02:00 - 2024-08-03T00:00:00.000+02:00 -> Insertando en columnas K y L, valores: 29.0, 8.0
Fecha: 2024-08-03T00:00:00.000+02:00 - 2024-08-04T00:00:00.000+02:00 -> Insertando en columnas P y Q, valores: 0.0, 0.0
Fecha: 2024-08-04T00:00:00.000+02:00 - 2024-08-05T00:00:00.000+02:00 -> Insertando en columnas U y V, valores: 0.0, 0.0
Fecha: 2024-08-05T00:00:00.000+02:00 - 2024-08-06T00:00:00.000+02:00 -> Insertando en columnas Z y AA, valores: 63.0, 37.0
Fecha: 2024-08-06T00:00:00.000+02:00 - 2024-08-07T00:00:00.000+02:00 -> Insertando en columnas AE y AF, valores: 46.0, 24.0
Fecha: 2024-08-07T00:00:00.000+02:00 - 2024-08-08T00:00:00.000+02:00 -> Insertando en columnas AJ y AK, valores: 58.0, 34.0
Fecha: 2024-08-08T00:00:00.000+02:00 - 2024-08-09T00:00:00.000+02:00